<a href="https://colab.research.google.com/github/ginots/Data-Science-Research/blob/main/Deep-Learning/world_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/My_api's/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d "puneet6060/intel-image-classification"

In [ ]:
!unzip /content/intel-image-classification.zip

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D,BatchNormalization, GlobalAveragePooling2D,Dropout,Activation
from tensorflow.keras .models import Sequential
model = Sequential([
    Input((150,150,3)),
    Conv2D(64,(3,3),activation=None),
    BatchNormalization(),
    Activation("relu"),
    MaxPooling2D((2,2)),
    Conv2D(128,(3,3),activation=None),
    BatchNormalization(),
    Activation("relu"),
    MaxPooling2D((2,2)),
    Conv2D(256,(3,3),activation=None),
    BatchNormalization(),
    Activation("relu"),
    MaxPooling2D((2,2)),
    GlobalAveragePooling2D(),
    Dense(128,activation="relu"),
    Dropout(0.5),
    Dense(6,activation="softmax"),
])

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
train_data = train_generator.flow_from_directory(
    "/content/seg_train/seg_train", target_size=(150,150), class_mode="sparse", subset="training"
    )
val_data = train_generator.flow_from_directory(
    "/content/seg_train/seg_train", target_size=(150,150), class_mode="sparse", subset="validation"
)

In [ ]:
train_data.class_indices

In [ ]:
val_data.class_indices

In [ ]:
model.fit(train_data, validation_data=val_data, epochs=10)

In [ ]:
model.save("world_classification.keras")

In [ ]:
from tensorflow.keras.models import load_model
final_model = load_model("/content/drive/MyDrive/models/world_classification.keras")

In [ ]:
final_model.summary()

In [ ]:
test_data = train_generator.flow_from_directory("/content/seg_test/seg_test", target_size=(150,150), class_mode="sparse", shuffle=False)

In [ ]:
test_data.class_indices

In [ ]:
y_test = test_data.classes

In [ ]:
final_model.evaluate(test_data)

In [ ]:
y_pred = final_model.predict(test_data)

In [ ]:
import numpy as np

In [ ]:
f_pred = np.argmax(y_pred,axis=1)

In [ ]:
f_pred

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,f_pred))

In [ ]:
import cv2

In [ ]:
train_data[0][0].shape

In [ ]:
def predict(path):
  img = cv2.imread(path)
  img_resized = cv2.resize(img,(150,150))
  img_reshaped = img_resized.reshape((1,150,150,3))/255
  res = final_model.predict(img_reshaped)
  f = np.argmax(res,axis=1)[0]
  classes = ["Buildings", "Forest", "Glacier", "Mountain", "Sea", "Street"]
  print(classes[f])




In [ ]:
predict("/content/seg_pred/seg_pred/10034.jpg")